## **Student's Autograder**
This is a project to calculate quiz and term scores from q&a pdf and response pdf.

Whats left: 

* Editing pdf

bugs:
* not able to parse programme feedback questions
* response pdf has issues, missing 1 question

### Importing the required libraries 

In [230]:
import os
import pandas as pd
import numpy as np
import tabula
from ipywidgets import widgets
from PyPDF2 import PdfFileReader, PdfFileWriter
from PyPDF2.generic import DecodedStreamObject, EncodedStreamObject,ArrayObject

In [231]:
#debugging for notebooks current directory for reading files later.
import os
os.getcwd()

'/home/hem436/Documents/TDS/Week 2'

### Reading pdfs

In [232]:
resp_pdf = widgets.Text(
 placeholder='Enter filename with extension',
 description='Response pdf:'
 )
qa_pdf = widgets.Text(
 placeholder='Enter filename with extension',
 description='Q&A pdf:'
 )
#POD22S1C11500062.pdf
#IIT M DAD DIPLOMA AN2 EXAM QPD1.pdf
display(resp_pdf,qa_pdf)

Text(value='', description='Response pdf:', placeholder='Enter filename with extension')

Text(value='', description='Q&A pdf:', placeholder='Enter filename with extension')

In [443]:
df = tabula.read_pdf(resp_pdf.value,pages='all',multiple_tables=False)
filename=qa_pdf.value

In [444]:
with pd.option_context("display.max_rows",None):
    print(df[0])

                 Name                     SHEKHAR MOHANTY HEMANT
0              QP Set  2022 Nov: IIT M DAD DIPLOMA AN2 EXAM QPD1
1    Test Center Name            iON Digital Zone iDZ Jamshedpur
2               Shift                                         AN
3             Roll No                           POD22S1C11500062
4         Question ID                           Options Selected
5        640653445429                                 Unanswered
6        640653445430                                 Unanswered
7        640653445431                                 Unanswered
8        640653445432                                 Unanswered
9        640653445433                                 Unanswered
10       640653445434                                 Unanswered
11       640653445435                                 Unanswered
12       640653445436                                 Unanswered
13       640653445437                                 Unanswered
14       640653445438    

### Variables initialization

In [445]:
my_question="Question ID"
my_answer="Options Selected"
unanswered="Unanswered"
file_qid="Question Id"

In [446]:
df[0].iloc[4].values

array(['Question ID', 'Options Selected'], dtype=object)

In [447]:
df[0].head()

,Name,SHEKHAR MOHANTY HEMANT
0,QP Set,2022 Nov: IIT M DAD DIPLOMA AN2 EXAM QPD1
1,Test Center Name,iON Digital Zone iDZ Jamshedpur
2,Shift,AN
3,Roll No,POD22S1C11500062
4,Question ID,Options Selected


In [448]:
df1=df[0].replace(unanswered,np.nan,regex=True)

Students response for exam dataframe

In [449]:
myans=df1.dropna()
myans.columns=myans.iloc[4].values
myans=myans[5:]

In [450]:
myans[my_question]=myans[my_question].apply(int)
myans=myans.sort_values(my_question)
print(len(myans))
myans

21


,Question ID,Options Selected
185,640653445629,6406531484861
186,640653445630,6406531484865
187,640653445631,6406531484870
188,640653445632,6406531484871
189,640653445633,6406531484873
190,640653445634,6406531484881
192,640653445635,6406531484882
191,640653445636,6406531484888
193,640653445637,6406531484889
195,640653445638,"6406531484891,6406531484892"


### Useful functions

In [451]:
def gentextarr(content):
    """
    Returns an array of text block properties of pdf
    Parameter:
        content: string object of page content obtained by page.getContents().getData().decode() 
    """
    lines = content.splitlines()
    result = []
    in_text = False
    for line in lines:
        if line == "BT":
            in_text = True
            temp=[]

        elif line == "ET":
            in_text = False
            result.append(temp)
            temp=[]

        elif in_text:
            if line[-2:].lower()=="tj" or line[-2:].lower()=="tf" or line[-2:].lower()=="rg":
                temp.append(line[:])
    return result

def gen_cmap(font,pg):
    """
    Returns a character map dictionary of a pdf 'filename' page 'pg' and font 
    Parameter:
     font: font key name in meta properties of pdf.
     pg: int page number of pdf
    """
    # pdf = PdfFileReader(filename)
    # page = pdf.getPage(pg)
    page=pg
    temp=False
    cmap={}
    for i in (page.getObject()['/Resources']['/Font'][font]['/ToUnicode'].getObject().getData().decode().split('\n')):
        if "endbfrange" in i:
            temp=False
        if temp:
            cmap[i[1:5]]=i[-5:-1]
        if "beginbfrange" in i:
            temp=True
    return cmap

def font_decoder(text,cmap):
    """
    Returns decoded word from pdf text hex code. 
    """

    count=0
    word=""
    temp=""
    for i in text:
        count+=1
        if count==4:
            temp+=i
            word+=chr(int(cmap.get(temp.lower(),temp.lower()),16))
            temp=""
            count=0
        else:
            temp+=i
    return word


## PDF Content

In [452]:
"""
This piece of code extracts question answer pdf text content as well as correct answer key.
"""
pdf = PdfFileReader(filename)
sum=0
df=[[file_qid,"Correct Marks","Answers"]]
m_bool=False
marks=""
ans_bool=False
ans=""
row=[]
temp=[]
s_head=[]
temp_head=""
ishead=False
for p in range(0,pdf.getNumPages()):
    content=pdf.pages[p].getContents()
    #a different type content
    if type(content)==ArrayObject:
        m=""
        for n in content:
            m+=n.getObject().getData().decode()
        content=m
    elif type(content)==EncodedStreamObject or type(content)==DecodedStreamObject:
        content=content.getData().decode()
    for i in gentextarr(content):
        text=""
        ans_bool=False
        for j in i:
            # getting character map for a font type
            if j[-2:].lower()=="tf":
                cmap=gen_cmap(j.split()[0],pdf.pages[p])
                #checking for subject headings with font size =18
                if j.split()[1]=="18":
                    ishead=True
            # checking green color as ind of correctness        
            if j=="0 0.50196 0 rg" and len(row)>1:
                ans_bool=True
    
            elif j[-2:].lower()=="tj":
                text=font_decoder(j[1:-3],cmap)
                if ishead:
                    ishead=False
                    temp_head=text
                
                if (not m_bool) and (not ans_bool):
                    # checking for question id
                    if str(file_qid) in text:
                        if len(temp)>0:
                            row.append(temp)
                            df.append(row)
                        row=[]
                        temp=[]
                        if text.split()[7].isnumeric():
                            s_head.append(temp_head)
                            row=[text.split()[7]]
                            m_bool=True
                    
                # extracting marks
                elif m_bool:
                    if "Correct Marks" in text:
                        # appending to heading list if marks is 0
                        # if text.split()[-1]=="0":
                        #     s_head.append(temp_head)
                        if len(row)==1:
                            row.append(text.split()[3])
                        m_bool=False
                        
                #extracting answer key
                elif ans_bool:
                    if text!=" ":
                        if len(text)>=13 and text[:13].isnumeric():
                            temp.append(text[:13])
                        elif len(text.split())==3 and "to" in text.split()[1]:
                            t=text.split()
                            temp=[float(t[0]),float(t[-1])]
                        elif len(temp)==0:
                            temp.append(text)
if len(temp)>=1:
    row.append(temp)
    df.append(row)

In [453]:
print(len(df))
df[1]

278


['640653445429', '0.5', ['6406531484269']]

In [454]:
df2=pd.DataFrame(df[1:],columns=df[0])
df2[file_qid]=df2[file_qid].apply(int)
df2["Subjects"]=np.array(s_head)
df2=df2.sort_values([file_qid])
df2.head()

,Question Id,Correct Marks,Answers,Subjects
0,640653445429,0.5,[6406531484269],Programme Feedback
1,640653445430,0.5,[6406531484272],Programme Feedback
2,640653445431,0.5,[6406531484274],Programme Feedback
3,640653445432,0.5,[6406531484279],Programme Feedback
4,640653445433,0.5,[6406531484284],Programme Feedback


In [455]:
for i in set(s_head):
    print(i,s_head.count(i))

CT 18
Java 16
MLT 22
Maths2 15
BDM 16
System commands 17
Statistics2 18
MLP 25
PDSA 16
AppDev1 19
AppDev2 17
MLF 15
Business Analytics 15
DBMS 17
Programme Feedback 31


In [457]:
def evaluate(df2):
    
    #Multi option
    df2['Score'] = df2.apply(lambda x: (len(set(x["My answer"].split(","))\
                                            .intersection(set(x["Answers"])))/len(x["Answers"]))*float(x['Correct Marks'])\
                             if type(x["Answers"][0])==str and len(x["Answers"][0])==13 and\
                                len(set(x["My answer"].split(",")))<=len(set(x["Answers"])) and\
                                set(x["My answer"].split(",")).issubset(set(x["Answers"]))\
                             else x["Score"], axis=1)
    #integer ranged
    df2['Score'] = df2.apply(lambda x: float(x['Correct Marks'])\
                             if len(x["My answer"])<13 and len(x["My answer"])>0 and\
                                len(x['Answers'])>=1 and\
                                float(x['My answer']) >= float(x["Answers"][0]) and\
                                float(x['My answer']) <= float(x["Answers"][-1])\
                             else x["Score"], axis=1)
    return df2

## Final score

### Method 1: using index of 0 correct marks

In [458]:
# s_index=df3[df3["Correct Marks"]=="0"].index
# s_index

In [459]:
# subjects={}
# for i in range(len(s_index)):
#     if i==len(s_index)-1:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:]
#     else:
#         subjects[df3["Subjects"][s_index[i]]]=df3.iloc[s_index[i]:s_index[i+1]]
# subjects.keys()

In [460]:
# reports=[]
# for i in subjects.values():
#     print(i["Correct Marks"].apply(float).values)
#     reports.append([i["Subjects"].unique()[0],i["Correct Marks"].apply(float).values.sum(),i["Score"].sum()])

In [461]:
# for i in reports:
#     print(i[0],":",i[2]/i[1]*100,"%")

### Method 2: Using groupby function

In [462]:
df3=df2.set_index('Question Id').combine_first(myans.set_index('Question ID'))\
       .reset_index().replace(np.NAN,"")
df3.columns=["Question Id","Answers","Correct Marks","My answer","Subjects" ]
df3["Score"]=np.zeros(df3.shape[0])
df3=evaluate(df3)


In [463]:
df3[df3["Subjects"]=="MLT"]

,Question Id,Answers,Correct Marks,My answer,Subjects,Score
175,640653445621,"[6406531484830, 6406531484831]",2,,MLT,0.0
183,640653445629,[6406531484861],0,6406531484861,MLT,0.0
184,640653445630,[6406531484865],7,6406531484865,MLT,7.0
185,640653445631,[6406531484870],6,6406531484870,MLT,6.0
186,640653445632,[6406531484871],6,6406531484871,MLT,6.0
187,640653445633,[6406531484876],6,6406531484873,MLT,0.0
188,640653445634,[6406531484881],6,6406531484881,MLT,6.0
189,640653445635,[6406531484882],4,6406531484882,MLT,4.0
190,640653445636,[6406531484888],6,6406531484888,MLT,6.0
191,640653445637,[6406531484889],5,6406531484889,MLT,5.0


In [464]:
df3["Correct Marks"]=df3["Correct Marks"].apply(float)

In [465]:
report2=pd.DataFrame(df3.groupby("Subjects")[["Correct Marks","Score"]].sum())

In [466]:
report2["Percentage"]=(report2["Score"]*100/report2["Correct Marks"]).round(2)

In [467]:
report2.columns=["Total Marks","Scored","Percentage"]
headers = {
    'selector': 'thead',
    'props': 'background-color: #0dcaf0; color: white;'
}
report2.style.applymap(lambda v: 'color:red;' if (v == 0) else 'color:green',subset=["Percentage","Scored"])\
    .set_table_styles([headers])\
    .format({"Total Marks":'{:.2f}',"Scored":'{:.2f}',"Percentage":'{0:,.2f}%'})

,Total Marks,Scored,Percentage
Subjects,,,
AppDev1,49.50,0.00,0.00%
AppDev2,51.00,0.00,0.00%
BDM,18.00,0.00,0.00%
Business Analytics,20.00,0.00,0.00%
CT,51.00,0.00,0.00%
DBMS,51.00,0.00,0.00%
Java,50.00,0.00,0.00%
MLF,51.50,0.00,0.00%
MLP,52.00,5.00,9.62%


In [468]:
report2[report2["Percentage"]>0]

,Total Marks,Scored,Percentage
Subjects,,,
MLP,52.0,5.0,9.62
MLT,97.0,82.0,84.54


### Exporting dataframe as csv

In [ ]:
df3[df3["Subjects"].isin(report2[report2["Percentage"]>0].index)].to_csv("summary.csv",index=False)
report2[report2["Percentage"]>0].to_csv("report_card.csv")

In [469]:
s_head

['Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Programme Feedback',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Maths2',
 'Statistics2',
 'Statistics2',
 'Statistics2',
 'Statistics2',
 'Statistics2',
 'Statistics2',
 'Statistics2',
 'Statisti